# Imports et chargement des données

In [5]:
import faiss
import numpy as np
import pandas as pd

# 📁 Chemins
embedding_path = "../data/embeddings/chunks_embeddings.npy"
metadata_path = "../data/embeddings/chunks_metadata.csv"

# Chargement
embeddings = np.load(embedding_path)
df_meta = pd.read_csv(metadata_path)

print(f"{embeddings.shape[0]} vecteurs chargés.")
print(f"{df_meta.shape[0]} métadonnées chargées.")


3316 vecteurs chargés.
3316 métadonnées chargées.


# Création de l’index FAISS

In [6]:
# Dimension des vecteurs
d = embeddings.shape[1]

# Index simple (Flat L2)
index = faiss.IndexFlatL2(d)

# Ajout des vecteurs
index.add(embeddings)

print(f"✅ Index FAISS créé avec {index.ntotal} vecteurs.")


✅ Index FAISS créé avec 3316 vecteurs.


# Sauvegarde de l’index

In [7]:
faiss.write_index(index, "../data/embeddings/faiss_index.idx")
print("📦 Index sauvegardé.")


📦 Index sauvegardé.


# Test de requête sémantique

In [8]:
from sentence_transformers import SentenceTransformer

# Recharger le modèle d'embedding
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Exemple de requête
query = "Comment reconnaître la jalousie chez quelqu’un ?"
query_vec = model.encode([query])

# Recherche dans l’index
k = 5  # nombre de résultats
D, I = index.search(query_vec, k)

# Affichage des résultats
for i in I[0]:
    print(f"\n🔹 Chunk ID: {df_meta.iloc[i]['chunk_id']}")
    print(f"📘 Source: {df_meta.iloc[i]['source']}")
    print(f"📄 Page: {df_meta.iloc[i]['page']}")
    print(f"🏷️ Thème: {df_meta.iloc[i]['theme']}")
    print(f"📝 Texte:\n{df_meta.iloc[i]['text'][:500]}...")



🔹 Chunk ID: 100 idees pour enseigner les ha - Mehdi Liratni_p182_c1
📘 Source: 100 idees pour enseigner les ha - Mehdi Liratni
📄 Page: 182
🏷️ Thème: émotions
📝 Texte:
De plus, une personne qui valide les émotions avec une attitude compatissante donne vraiment envie de se confier et d’écouter ses conseils par la suite. • Autorisez les « sorties en amoureux » sous condition de connaître...

🔹 Chunk ID: 100 idees pour enseigner les ha - Mehdi Liratni_p181_c1
📘 Source: 100 idees pour enseigner les ha - Mehdi Liratni
📄 Page: 181
🏷️ Thème: relations sociales
📝 Texte:
L’empathie est souvent un bon moyen de ne pas étouffer l’autre. Nous les encouragerons aussi à développer des centres d’intérêt (activités) solitaires pour qu’ils ne se sentent pas dépendants des autres. D’autres jeunes, au contraire, par crainte du lien affectif, vont plutôt se comporter de manière « évitante » et ne parviendront pas à créer d’intimité par crainte d’être trahis ou déçus. En les félicitant pour leur prudence, il

# Suite du test de la recherche sémantique - plusieures questions 

In [42]:
from sentence_transformers import SentenceTransformer

# Recharger le modèle
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Liste des questions à tester
queries = [
    "Comment réagir quand quelqu’un me fait une remarque blessante ?",
    "Pourquoi certaines personnes évitent le contact visuel ?",
    "Qu’est-ce que la joie et comment la reconnaître chez soi ?",
    "Comment se faire des amis quand on est timide ou différent ?",
    "Est-ce normal de ne pas comprendre les blagues ?"
]

# Nombre de résultats par requête
k = 5

# Boucle sur les requêtes
for query in queries:
    print(f"\n🔍 Question : {query}")
    query_vec = model.encode([query])
    D, I = index.search(query_vec, k)

    for rank, i in enumerate(I[0]):
        chunk = df_meta.iloc[i]
        print(f"\n  🔹 Résultat {rank + 1}")
        print(f"  📘 Source : {chunk['source']}")
        print(f"  📄 Page : {chunk['page']}")
        print(f"  🏷️ Thème : {chunk['theme']}")
        print(f"  🎯 Type : {chunk['type']}")
        print(f"  📚 Niveau : {chunk['niveau']}")
        print(f"  📝 Texte :\n  {chunk['text'][:500]}...\n  ---")



🔍 Question : Comment réagir quand quelqu’un me fait une remarque blessante ?

  🔹 Résultat 1
  📘 Source : 100 idees pour enseigner les ha - Mehdi Liratni
  📄 Page : 220
  🏷️ Thème : relations sociales
  🎯 Type : inconnu
  📚 Niveau : intermédiaire
  📝 Texte :
  c’est une gare ou un métro), demander à un passant, appeler le service client en cherchant sur internet… – Si je suis agressé verbalement ou physiquement : crier « à l’aide, au secours », s’enfuir, se rapprocher d’un groupe et leur expliquer la situation, aller vers les employés (contrôleurs, chauffeurs, guichetiers…), appeler la police ou mes proches. – Si l’on me rackette avec une arme : je donne ce qu’on me demande, je demande de l’aide à un passant pour appeler la police ou mes proches, je m...
  ---

  🔹 Résultat 2
  📘 Source : Etre autiste et ado_ Strategies - Inconnu(e)
  📄 Page : 205
  🏷️ Thème : autisme
  🎯 Type : exemple
  📚 Niveau : intermédiaire
  📝 Texte :
  TRUCS POUR GÉRER L’INTIMIDATION Un bon outil pour gérer l’

# Vectorstore FAISS

A présent, nous allons créer le vectorstore FAISS avec LangChain ce qui est necessaire pour l'utiliser dans le pipeline RAG. Le vectorstore FAISS est compatible avec LangChain et il permet d'utiliser ses fonctionnalités pour la recherche et la récupération d'informations. La création de vectorstore FAISS facilite également l'intégration avec d'autres composants de LangChain pour construire des applications de NLP avancées.

In [9]:
# Créer le vectorstore LangChain avec les documents et l'index FAISS sauvegardé pour l'utiliser dans le pipeline RAG

from langchain.vectorstores import FAISS
from langchain.schema import Document

# Recréer les documents à partir des métadonnées
documents = [
    Document(
        page_content=row["text"],
        metadata={
            "chunk_id": row["chunk_id"],
            "source": row["source"],
            "page": row["page"],
            "theme": row["theme"],
            "niveau": row["niveau"],
            "type": row["type"]
        }
    )
    for _, row in df_meta.iterrows()
]


In [10]:
# Importer les variables d'environnement (ici la clé API OpenAI)
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


In [11]:
# Importer langchain si pas déjà fait pour utiliser les embeddings OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# Charger la clé API
embedding_function = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Créer le vectorstore avec les embeddings pré-calculés et les documents pour le pipeline RAG
vectorstore = FAISS.from_documents(
    documents,
    embedding_function
)



C:\Users\aheij\AppData\Local\Temp\ipykernel_3976\3921327780.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_function = OpenAIEmbeddings(openai_api_key=openai_api_key)


In [12]:
# Sauvegarder le vectorstore pour l'utiliser dans le pipeline RAG

# 📁 Dossier de sauvegarde
vectorstore.save_local("../data/embeddings/faiss_store")
print("✅ Vectorstore sauvegardé avec .idx et .pkl")


✅ Vectorstore sauvegardé avec .idx et .pkl


Différence entre FAISS index et vectorstore:

Ces deux termes sont liés, mais ils ne désignent pas exactement la même chose :

FAISS index: c'est la structure de données optimisée pour la recherche de similarité entre vecteurs. [index.faiss stocke les vecteurs dans un format rapide à interroger].

Vectorstore: c'est une abstraction LangChain qui combine le FAISS index + les chunks + les métadonnées.	[vectorstore = FAISS.load_local(...) te donne accès aux chunks et à leurs sources].

En résumé : FAISS index = moteur brut, vectorstore = moteur + contenu + contexte.